# Bước 1: Import thư viện

In [14]:
import pandas as pd #đọc dữ liệu
import numpy as np #xử lý dữ liệu
import math
import datetime as dt
import matplotlib.pyplot as plt #vẽ biểu đồ
import seaborn as sb

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.layers import LSTM, GRU
from tensorflow.keras.models import load_model #tải mô hình
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten

# Các lớp để xây dựng mô hình
from keras.models import Sequential #đầu vào
from keras.layers import LSTM 
from keras.layers import SimpleRNN
from keras.layers import Dropout #tránh học tủ
from keras.layers import Dense #đầu ra
from keras.callbacks import ModelCheckpoint #lưu lại huấn luyện tốt nhất

# Kiểm tra độ chính xác của mô hình
from sklearn.metrics import r2_score #đo mức độ phù hợp
from sklearn.metrics import mean_squared_error, mean_absolute_error, explained_variance_score, r2_score  
from sklearn.metrics import mean_absolute_percentage_error #đo % sai số tuyệt đối trung bình
from sklearn.metrics import mean_poisson_deviance, mean_gamma_deviance, accuracy_score
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split 

from itertools import cycle
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots

# Bước 2: Đọc dữ liệu

In [16]:
# đọc dữ liệu từ file csv
df = pd.read_csv('Dữ liệu Lịch sử VNM 2013_2023.csv')

In [ ]:
df

In [18]:
# Xóa hai dòng "KL" và "Thay đổi %" từ DataFrame dataSet
df = df.drop(columns=["KL", "% Thay đổi"])

In [ ]:
# Hiển thị lại DataFrame sau khi xóa

df

# Bước 3: Mô tả dữ liệu

In [20]:
#định dạng cấu trúc thời gian
df["Ngày"] = pd.to_datetime(df.Ngày,format="%d/%m/%Y")

In [21]:
#kích thước dữ liệu
df.shape

(2505, 5)

In [ ]:
#dữ liệu 5 dòng đầu
df.head()

In [ ]:
#xác định kiểu dữ liệu
df.info()

In [ ]:
# Xem khoảng thời gian của bộ dữ liệu
print("Ngày đầu tiên trong bộ dữ liệu: ", df.iloc[-1][0])
print("Ngày cuối cùng trong bộ dữ liệu: ",df.iloc[0][0])
print("Khoảng thời gian: ", df.iloc[0][0]-df.iloc[-1][0])

In [ ]:
#mô tả bộ dữ liệu
df.describe()

# Bước 4: Tiền xử lý dữ liệu

In [ ]:
from matplotlib.dates import YearLocator, DateFormatter, MonthLocator # Thêm MonthLocator vào để sửa lỗi

# Chuyển đổi cột "Ngày" sang dạng datetime
df['Ngày'] = pd.to_datetime(df['Ngày'], format='%d/%m/%Y')

# Sắp xếp lại dữ liệu theo thứ tự thời gian
df = df.sort_values(by='Ngày')

# Chuyển đổi định dạng các cột giá thành số thực
df['Đóng cửa'] = df['Đóng cửa'].str.replace(',', '').astype(float)
df['Mở cửa'] = df['Mở cửa'].str.replace(',', '').astype(float)
df['Cao nhất'] = df['Cao nhất'].str.replace(',', '').astype(float)
df['Thấp nhất'] = df['Thấp nhất'].str.replace(',', '').astype(float)

# Lấy thông tin năm từ cột "Ngày"
df['Năm'] = df['Ngày'].dt.year

# Tạo đồ thị giá đóng cửa qua các năm
plt.figure(figsize=(10, 5))
plt.plot(df['Ngày'], df['Đóng cửa'], label='Giá đóng cửa', color='red')
plt.xlabel('Năm')
plt.ylabel('Giá đóng cửa')
plt.title('Biểu đồ giá đóng cửa của VNM qua các năm')
plt.legend(loc='best')

# Định dạng đồ thị hiển thị các ngày tháng theo năm-tháng
years = YearLocator()
yearsFmt = DateFormatter('%Y')
months = MonthLocator()  # Thêm dòng này để khai báo MonthLocator
plt.gca().xaxis.set_major_locator(years)
plt.gca().xaxis.set_major_formatter(yearsFmt)
plt.gca().xaxis.set_minor_locator(months)

plt.tight_layout()
plt.show()


In [ ]:
features = ['Mở cửa', 'Đóng cửa', 'Cao nhất', 'Thấp nhất']
 
plt.subplots(figsize=(20,10))
 
for i, col in enumerate(features):
  plt.subplot(2,3,i+1)
  sb.distplot(df[col])
plt.show()

Trong biểu đồ phân phối dữ liệu OHLC, chúng ta có thể thấy hai đỉnh có nghĩa là dữ liệu đã thay đổi đáng kể ở hai vùng. Và dữ liệu có xu hướng bị lệch trái.

In [28]:
df1 = pd.DataFrame(df,columns=['Ngày','Đóng cửa'])
df1.index = df1.Ngày
df1.drop('Ngày',axis=1,inplace=True)
df1

,Đóng cửa
Ngày,
2013-07-10,94444.0
2013-07-11,94444.0
2013-07-12,95139.0
2013-07-15,95139.0
2013-07-16,96528.0
...,...
2023-07-17,72300.0
2023-07-18,72400.0
2023-07-19,72600.0


## Chia tập dữ liệu

In [31]:
df1.head()

,Đóng cửa
Ngày,
2013-07-10,94444.0
2013-07-11,94444.0
2013-07-12,95139.0
2013-07-15,95139.0
2013-07-16,96528.0


In [ ]:

data = df1['Đóng cửa'] 
  
# using the train test split function 
train_data, test_data = train_test_split(data , 
								random_state=104, 
								test_size=0.2, 
								shuffle=True) 

# printing out train and test sets 

print('data_train : ') 
print(train_data.head()) 
print(train_data.shape)
print('') 
print('data_test : ') 
print(test_data.head())
print(test_data.shape) 
print('') 


## Chuẩn hóa dữ liệu

In [35]:
sc = MinMaxScaler(feature_range=(0,1))
sc_train = sc.fit_transform(df1.values)

## Tạo tập x_train và y_train

In [36]:
x_train,y_train=[],[]
for i in range(50,len(train_data)):
  x_train.append(sc_train[i-50:i,0]) # giá đóng cửa 50 ngày cuối cùng trong tệp dữ liệu
  y_train.append(sc_train[i,0]) #lấy ra giá đóng cửa ngày hôm sau

In [ ]:
x_train #x_train là một chuỗi các giá trị đóng cửa liên tiếp trong quá khứ.

In [ ]:
y_train #y_train chứa giá trị đóng cửa  tiếp theo mà mô hình dự đoán. 

In [39]:
#xếp dữ liệu thành 1 mảng 2 chiều
x_train = np.array(x_train)
y_train = np.array(y_train)

In [40]:
#xếp lại dữ liệu thành mảng 1 chiều
x_train = np.reshape(x_train,(x_train.shape[0],x_train.shape[1],1))
y_train = np.reshape(y_train,(y_train.shape[0],1))

# Bước 5: Xây dựng và huấn luyện mô hình

## 5.1.MÔ HÌNH RNN

### Xây dựng mô hình RNN

In [ ]:
regressor = Sequential()

# Lớp thứ nhất của RNN
regressor.add(
    SimpleRNN(units = 50,
              activation = "tanh", # Hàm kích hoạt được sử dụng là tanh
              return_sequences = True, # Layer này sẽ trả về chuỗi đầu ra cho mỗi bước trong chuỗi đầu vào.
              input_shape = (x_train.shape[1],1))
              # Xác định kích thước của đầu vào. Trong trường hợp này, đầu vào có kích thước là (số lượng bước thời gian, số chiều của mỗi bước thời gian), với số chiều là 1.
             )

regressor.add(
    Dropout(0.2)
             )

# Lớp thứ hai RNN
regressor.add(
    SimpleRNN(units = 50,
              activation = "tanh",
              return_sequences = True)
             )

regressor.add(
    Dropout(0.2)
             )

# Lớp thứ ba RNN
regressor.add(
    SimpleRNN(units = 50,
              activation = "tanh",
              return_sequences = True)
             )

regressor.add(
    Dropout(0.2)
             )

# Lớp thứ tư RNN
regressor.add(
    SimpleRNN(units = 50)
             )

regressor.add(
    Dropout(0.2)
             )

# Thêm lớp đầu ra
regressor.add(Dense(units = 1))

# Đo sai số giá trị trung bình của bình phương chênh lệch giữa dự đoán và giá trị thực tế, có sử dụng trình tối ưu hóa adam
regressor.compile(
    optimizer = "adam",
    loss = "mse",
    metrics = ["accuracy"])

### Huấn luyện dữ liệu 

In [ ]:
save_rnn_model = "save_model.hdf5"
best_rnn_model = ModelCheckpoint(save_rnn_model,monitor='loss',verbose=2,save_best_only=True,mode='auto')
history = regressor.fit(x_train,y_train,epochs=50,batch_size=50,verbose=2,callbacks=[best_rnn_model])

### Kiểm tra và sử dụng mô hình

In [ ]:
plt.figure(figsize =(10,5))
plt.plot(history.history["accuracy"])
#Accuracy là tỉ lệ phần trăm của số lượng dự đoán đúng trên tổng số dự đoán. Nó là một phép đo quan trọng để đánh giá hiệu suất của mô hình trên dữ liệu.
plt.xlabel("Epochs")
plt.ylabel("Accuracies")
plt.title("Mô hình RNN, Accuracy và Epoch")
plt.show()

In [ ]:
#Mô hình chỉ số loss
plt.figure(figsize =(10,7))
plt.plot(history.history["loss"])
#Loss là một số đo đánh giá mức độ sai lệch giữa giá trị dự đoán của mô hình và giá trị thực tế
plt.xlabel("Epochs")
plt.ylabel("Losses")
plt.title("Mô hình RNN, Loss và Epoch")
plt.show()


In [ ]:
#dữ liệu train
y_train = sc.inverse_transform(y_train) #giá thực
final_model = load_model("save_model.hdf5")
rnn_train_predict = final_model.predict(x_train) #dự đoán giá đóng cửa trên tập đã train
rnn_train_predict = sc.inverse_transform(rnn_train_predict) #giá dự đoán

In [ ]:
#xử lý dữ liệu test
test_rnn = df1[len(train_data)-50:].values
test_rnn = test_rnn.reshape(-1,1)
rnn_sc_test = sc.transform(test_rnn)

x_test_rnn = []
for i in range(50,test_rnn.shape[0]):
  x_test_rnn.append(rnn_sc_test[i-50:i,0])
x_test_rnn = np.array(x_test_rnn)
x_test_rnn = np.reshape(x_test_rnn,(x_test_rnn.shape[0],x_test_rnn.shape[1],1))

#dữ liệu test
y_test_rnn = data[1954:] #giá thực
y_test_predict_rnn = final_model.predict(x_test_rnn)
y_test_predict_rnn = sc.inverse_transform(y_test_predict_rnn) #giá dự đoán

# Kiểm tra và cắt bớt mẫu
min_length = min(len(y_test_rnn), len(y_test_predict_rnn))
y_test_rnn = y_test_rnn[:min_length]
y_test_predict_rnn = y_test_predict_rnn[:min_length]


In [ ]:
#lập biểu đồ so sánh
train_data_rnn = df1[50:2004]
test_data_rnn = df1[2004:]

plt.figure(figsize=(10,5))
plt.plot(df1,label='Giá thực tế',color='red') #đường giá thực
train_data_rnn['Dự đoán'] = rnn_train_predict #thêm dữ liệu
plt.plot(train_data_rnn['Dự đoán'],label='Giá dự đoán train',color='green') #đường giá dự báo train
test_data_rnn['Dự đoán'] = y_test_predict_rnn #thêm dữ liệu
plt.plot(test_data_rnn['Dự đoán'],label='Giá dự đoán test',color='blue') #đường giá dự báo test
plt.title('So sánh giá dự báo và giá thực tế') #đặt tên biểu đồ
plt.xlabel('Năm') #đặt tên hàm x
plt.ylabel('Giá đóng cửa (VNĐ)') #đặt tên hàm y
plt.legend() #chú thích
plt.show()

In [ ]:
# Các chỉ số đánh giá sự hiệu quả trong dự đoán của mô hình dựa trên tập train

print('Độ phù hợp tập train:',r2_score(y_train,rnn_train_predict))

print('Sai số tuyệt đối trung bình trên tập train (VNĐ):',mean_absolute_error(y_train,rnn_train_predict))

print('Phần trăm sai số tuyệt đối trung bình tập train:',mean_absolute_percentage_error(y_train,rnn_train_predict))

In [ ]:
# Bảng giá dự đoán dựa trên tập train
train_data_rnn

In [ ]:
# Các chỉ số đánh giá sự hiệu quả trong dự đoán của mô hình dựa trên tập test

#r2
print('Độ phù hợp tập test:',r2_score(y_test_predict_rnn, y_test_rnn))

#mae
print('Sai số tuyệt đối trung bình trên tập test (VNĐ):',mean_absolute_error(y_test_rnn,y_test_predict_rnn))

#mape
print('Phần trăm sai số tuyệt đối trung bình tập test:',mean_absolute_percentage_error(y_test_rnn,y_test_predict_rnn))

In [ ]:
# Bảng giá dự đoán dựa trên tập test
test_data_rnn

## 5.2.MÔ HÌNH LSTM

### Xây dựng mô hình LSTM

In [ ]:
#Xây dựng mô hình

model = Sequential()
model.add(LSTM(units=128, input_shape = (x_train.shape[1],1), return_sequences=True))
model.add(LSTM(units=64))
model.add(Dropout(0,5))
model.add(Dense(1))
model.compile(loss = 'mean_absolute_error', optimizer = 'adam')

### Huấn luyện dữ liệu

In [ ]:
#Huấn luyện mô hình
save_model = 'save_model.hdf5'
best_model = ModelCheckpoint(save_model, monitor = 'loss', verbose = 2, save_best_only=True, mode = 'auto')
history2 = model.fit(x_train, y_train, epochs=50, batch_size=50, verbose=2, callbacks=[best_model])

## Kiểm tra và sử dụng mô hình

Vẽ biểu đồ (plot) giá trị của độ đo "accuracy" trên tập huấn luyện qua các epoch (vòng lặp huấn luyện). Biểu đồ accuracy cung cấp cái nhìn quan trọng về hiệu suất của mô hình và giúp quyết định liệu là mô hình đã đủ tốt để triển khai hay chưa.

In [ ]:
plt.figure(figsize =(10,7))
plt.plot(history2.history["loss"]) 
#Loss là một số đo đánh giá mức độ sai lệch giữa giá trị dự đoán của mô hình và giá trị thực tế
plt.xlabel("Epochs")
plt.ylabel("Losses")
plt.title("Mô hình LSTM, Loss và Epoch")
plt.show()

In [ ]:
#Dữ liệu train
y_train = sc.inverse_transform(y_train)  # giá thực
final_model = load_model('save_model.hdf5')
y_train_predict = final_model.predict(x_train)
y_train_predict = sc.inverse_transform(y_train_predict)  # gía dự đoán

In [ ]:
#xử lý dữ liệu test
test = df1[len(train_data)-50:].values
test = test.reshape(-1,1)
sc_test = sc.transform(test)

x_test = []
for i in range(50,test.shape[0]):
  x_test.append(sc_test[i-50:i,0])
x_test = np.array(x_test)
x_test = np.reshape(x_test,(x_test.shape[0],x_test.shape[1],1))

#dữ liệu test
y_test = data[1954:] #giá thực
y_test_predict = final_model.predict(x_test)
y_test_predict = sc.inverse_transform(y_test_predict) #giá dự đoán

# Kiểm tra và cắt bớt mẫu 
min_length = min(len(y_test), len(y_test_predict))
y_test = y_test[:min_length]
y_test_predict = y_test_predict[:min_length]


In [ ]:
#lập biểu đồ so sánh
train_data1 = df1[50:2004]
test_data1 = df1[2004:]

plt.figure(figsize=(24,8))
plt.plot(df1,label='Giá thực tế',color='red') #đường giá thực
train_data1['Dự đoán'] = y_train_predict #thêm dữ liệu
plt.plot(train_data1['Dự đoán'],label='Giá dự đoán train',color='green') #đường giá dự báo train
test_data1['Dự đoán'] = y_test_predict #thêm dữ liệu
plt.plot(test_data1['Dự đoán'],label='Giá dự đoán test',color='blue') #đường giá dự báo test
plt.title('So sánh giá dự báo và giá thực tế') #đặt tên biểu đồ
plt.xlabel('Năm') #đặt tên hàm x
plt.ylabel('Giá đóng cửa (VNĐ)') #đặt tên hàm y
plt.legend() #chú thích
plt.show()

In [ ]:
print('Độ phù hợp của tập train:', r2_score(y_train,y_train_predict))
print('Sai số tuyệt đối trung bình của tập train:', mean_absolute_error(y_train, y_train_predict))
print('Phần trăm sai số tuyệt đối trung bình tập train:', mean_absolute_percentage_error(y_train, y_train_predict))

In [ ]:
train_data1

In [ ]:
print('Độ phù hợp của tập test:', r2_score(y_test,y_test_predict))
print('Sai số tuyệt đối trung bình của tập test:', mean_absolute_error(y_test, y_test_predict))
print('Phần trăm sai số tuyệt đối trung bình tập test:', mean_absolute_percentage_error(y_test, y_test_predict))

In [ ]:
test_data1

## 5.3.MÔ HÌNH GRU

### Xây dựng mô hình

In [ ]:
# # Chuẩn bị dữ liệu kiểm thử tương tự như dữ liệu huấn luyện
# x_test, y_test = [], []

# for i in range(50, len(test_data)):
#     x_test.append(sc_test[i - 50:i, 0])
#     y_test.append(sc_test[i, 0])

# # Chuyển đổi dữ liệu kiểm thử thành mảng
# x_test = np.array(x_test)
# y_test = np.array(y_test)

# # Reshape dữ liệu kiểm thử
# x_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1], 1))
# y_test = np.reshape(y_test, (y_test.shape[0], 1))
# Tạo dữ liệu test cho mô hình
x_test, y_test = [], []
for i in range(50, len(test_data)):
    x_test.append(sc_train[i - 50:i, 0])
    y_test.append(sc_train[i, 0])

x_test = np.array(x_test)
y_test = np.array(y_test)
x_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1], 1))
y_test = np.reshape(y_test, (y_test.shape[0], 1))


### Huấn luyện dữ liệu

In [ ]:
history3 = model.fit(x_train,y_train,validation_data=(x_test,y_test),epochs=10,batch_size=32,verbose=1)

### Kiểm tra và sử dụng mô hình

In [ ]:
# # Tạo dữ liệu test cho mô hình
# x_test, y_test = [], []
# for i in range(50, len(test_data)):
#     x_test.append(sc_train[i - 50:i, 0])
#     y_test.append(sc_train[i, 0])

# x_test = np.array(x_test)
# y_test = np.array(y_test)
# x_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1], 1))
# y_test = np.reshape(y_test, (y_test.shape[0], 1))

# Dự đoán giá đóng cửa trên tập test
predicted_stock_price = model.predict(x_test)
predicted_stock_price = sc.inverse_transform(predicted_stock_price)
y_test = sc.inverse_transform(y_test)

# Đánh giá mô hình
r2 = r2_score(y_test, predicted_stock_price)
mae = mean_absolute_error(y_test, predicted_stock_price)
mape = mean_absolute_percentage_error(y_test, predicted_stock_price)

# In các chỉ số đánh giá
print(f'Mức độ phù hợp: {r2}')
print(f'Độ sai số tuyệt đối trung bình của tập test: {mae}')
print(f'Phần trăm sai số tuyệt đối trung bình của tập test: {mape}')

# Hiển thị biểu đồ so sánh giá thực tế và giá dự đoán
plt.figure(figsize=(10, 5))
plt.plot(y_test, label='Actual Price', color='blue')
plt.plot(predicted_stock_price, label='Predicted Price', color='red')
plt.title('Actual vs Predicted Stock Price')
plt.xlabel('Time')
plt.ylabel('Stock Price')
plt.legend(loc='best')
plt.show()

In [ ]:
plt.figure(figsize =(10,5))
plt.plot(history3.history["loss"])
plt.xlabel("Epochs")
plt.ylabel("Losses")
plt.title("Mô hình GRU, Accuracy và Epoch")
plt.show()

## 5.4.MÔ HÌNH CNN

### Xây dựng mô hình CNN

In [ ]:
# Xây dựng mô hình CNN
model = Sequential()
model.add(Conv1D(filters=64, kernel_size=2, activation='relu', input_shape=(50, 1)))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(Dense(50, activation='relu'))
model.add(Dense(1, activation='linear'))

# Biên soạn mô hình
model.compile(optimizer='adam', loss='mean_squared_error')


### Huấn luyện mô hình

In [ ]:
history4 = model.fit(x_train, y_train, epochs=50, batch_size=32)

### Kiểm tra và sử dụng mô hình

In [ ]:
plt.figure(figsize =(10,7))
plt.plot(history4.history["loss"])
#Loss là một số đo đánh giá mức độ sai lệch giữa giá trị dự đoán của mô hình và giá trị thực tế
plt.xlabel("Epochs")
plt.ylabel("Losses")
plt.title("Mô hình CNN, Loss và Epoch")
plt.show()

In [ ]:
# Kiểm tra và sử dụng mô hình

# Chuẩn bị dữ liệu train
x_train, y_train = [], []
for i in range(50, len(train_data)):
    x_train.append(sc_train[i-50:i, 0])
    y_train.append(sc_train[i, 0])

x_train, y_train = np.array(x_train), np.array(y_train)

# Reshape dữ liệu train
x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1))

# Dự đoán giá trên tập train
predicted_prices_train = model.predict(x_train)
predicted_prices_train = sc.inverse_transform(predicted_prices_train)

# Chuyển ngược chuẩn hóa cho giá thực tế trên tập train
y_train_actual = sc.inverse_transform(y_train.reshape(-1, 1))



In [ ]:
# Đánh giá mô hình trên tập train
mape_train = mean_absolute_percentage_error(y_train_actual, predicted_prices_train)
mae_train = mean_absolute_error(y_train_actual, predicted_prices_train)
r2_train = r2_score(y_train_actual, predicted_prices_train)

print("Train Mean Absolute Percentage Error:", mape_train)
print("Train Mean Absolute Error:", mae_train)
print("Train R^2 Score:", r2_train)


In [ ]:
# Đánh giá mô hình trên tập test
mape_test = mean_absolute_percentage_error(y_test, y_test_predict)
mae_test = mean_absolute_error(y_test, y_test_predict)
r2_test = r2_score(y_test, y_test_predict)

print("Phần trăm sai số trung bình tập test:", mape_test)
print("Sai số tuyệt đối trung bình của tập test: ", mae_test)
print("Độ phù hợp của tập test:", r2_test)